# Classifer Model - Playground

### Evaluation Metric: __What?__

Using __what?__ as the evalation metric for the Classifer. 

There are 3 classes: Functional, Non Functional, and Functional Needs Repair. 

#### Class Frequency of the classes represented in the __Training__ dataset:
* Functional: 54.5%
* Non Functional: 38.7%
* Functional Needs Repair: 6.8%

There is a class imbalance issue, with __Functional Needs Repair__ as the rare class. 

### Challenge Evaluation Metric: Classification Rate 

Classification Rate = 1N∑Ni=0I(yi=yi^)

"The metric used for this competition is the classification rate, which calculates the percentage of rows where the predicted class y^
in the submission matches the actual class, y in the test set. The maximum is 1 and the minimum is 0. The goal is to maximize the classification rate."

#### Load in libraries

In [31]:
# Load in libraries
import warnings
from importlib import reload

warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np 

import scipy.stats as scs
import statsmodels.api as sm
import statsmodels.formula.api as sms

# ADD IN CORRECT CLASSIFIER MODEL LIBS!!!!!
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import KFold

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc # needed?
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, plot_confusion_matrix
#needed?
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
# needed?
from sklearn.model_selection import GridSearchCV, cross_val_score


import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn')

#### Load in Training Data

In [32]:
# read in the processed  training data
train_df = pd.read_csv('../data/train_processed_labeled.csv')
print(train_df.shape)
train_df.head(3)

(57565, 45)


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,basin,...,source_type,source_class,waterpoint_type,waterpoint_type_group,recorded_year,waterpoint_age,region_code_and_name,recorded_good_quality,recorded_good_quantity,status_group
0,69572,6000.0,2011-03-14,roman,1390,Roman,34.938093,-9.856322,none,lake nyasa,...,spring,groundwater,communal standpipe,communal standpipe,2011,12,iringa_11,True,True,functional
1,8776,0.0,2013-03-06,grumeti,1399,GRUMETI,34.698766,-2.147466,zahanati,lake victoria,...,rainwater harvesting,surface,communal standpipe,communal standpipe,2013,3,mara_20,True,False,functional
2,34310,25.0,2013-02-25,lottery club,686,World vision,37.460664,-3.821329,kwa mahundi,pangani,...,dam,surface,communal standpipe multiple,communal standpipe,2013,4,manyara_21,True,True,functional


In [33]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57565 entries, 0 to 57564
Data columns (total 45 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      57565 non-null  int64  
 1   amount_tsh              57565 non-null  float64
 2   date_recorded           57565 non-null  object 
 3   funder                  57565 non-null  object 
 4   gps_height              57565 non-null  int64  
 5   installer               57565 non-null  object 
 6   longitude               57565 non-null  float64
 7   latitude                57565 non-null  float64
 8   wpt_name                57565 non-null  object 
 9   basin                   57565 non-null  object 
 10  subvillage              57565 non-null  object 
 11  region                  57565 non-null  object 
 12  region_code             57565 non-null  int64  
 13  district_code           57565 non-null  int64  
 14  lga                     57565 non-null

In [34]:
train_df.describe()

,id,amount_tsh,gps_height,longitude,latitude,region_code,district_code,population,construction_year,recorded_year,waterpoint_age
count,57565.000000,57565.000000,57565.000000,57565.000000,57565.000000,57565.000000,57565.000000,57565.000000,57565.000000,57565.000000,57565.000000
mean,37108.796109,327.776129,689.600556,35.149066,-5.885342,15.223295,5.730600,185.644975,1996.875445,2011.915678,15.040858
std,21454.227195,3044.432384,693.565822,2.607774,2.810414,17.856558,9.761532,477.825269,10.225329,0.961137,10.257680
min,0.000000,0.000000,-90.000000,29.607122,-11.649440,1.000000,1.000000,0.000000,1960.000000,2002.000000,0.000000
25%,18528.000000,0.000000,0.000000,33.284575,-8.646677,5.000000,2.000000,0.000000,1995.000000,2011.000000,7.000000
50%,37057.000000,0.000000,427.000000,35.004942,-5.170124,12.000000,3.000000,35.000000,1997.000000,2012.000000,14.000000
75%,55669.000000,30.000000,1332.000000,37.234736,-3.372717,17.000000,5.000000,230.000000,2004.000000,2013.000000,17.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-0.998464,99.000000,80.000000,30500.000000,2013.000000,2013.000000,53.000000


In [36]:
train_df['amount_tsh'].value_counts()
train_df['population'].value_counts()
train_df['construction_year'].value_counts()
train_df['recorded_year'].value_counts()
train_df['waterpoint_age'].value_counts()

14    13914
15     5679
3      2740
1      2303
16     2246
2      2129
5      1981
4      1890
13     1869
7      1417
6      1381
11     1352
8      1160
33     1120
23      905
10      868
9       814
19      766
27      762
18      754
31      709
28      709
35      647
17      636
39      601
37      600
0       597
26      589
25      583
21      578
29      564
12      559
20      541
41      488
30      359
43      356
38      351
36      325
22      323
24      261
40      233
34      184
32      174
53       91
42       90
50       84
45       66
46       48
44       47
48       41
51       31
49       25
47       14
52       11
Name: waterpoint_age, dtype: int64

#### Load in Test Data

In [9]:
# read in the processed test data
test_df = pd.read_csv('../data/test_processed.csv')
print(test_df.shape)
test_df.head(3)

(14389, 44)


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,basin,...,source,source_type,source_class,waterpoint_type,waterpoint_type_group,recorded_year,waterpoint_age,region_code_and_name,recorded_good_quality,recorded_good_quantity
0,50785,0.0,2013-02-04,dmdd,1996,DMDD,35.290799,-4.059696,dinamu secondary school,internal,...,rainwater harvesting,rainwater harvesting,surface,other,other,2013,1,manyara_21,True,False
1,51630,0.0,2013-02-04,government of tanzania,1569,DWE,36.656709,-3.309214,kimnyak,pangani,...,spring,spring,groundwater,communal standpipe,communal standpipe,2013,13,arusha_2,True,False
2,17168,0.0,2013-02-01,unknown,1567,unknown,34.767863,-5.004344,puma secondary,internal,...,rainwater harvesting,rainwater harvesting,surface,other,other,2013,3,singida_13,True,False


In [26]:
test_df.describe()

,id,amount_tsh,gps_height,longitude,latitude,region_code,district_code,population,construction_year,recorded_year,waterpoint_age
count,14389.000000,14389.000000,14389.000000,14389.000000,14389.000000,14389.000000,14389.000000,14389.000000,14389.000000,14389.000000,14389.000000
mean,37164.400028,333.169831,676.137466,35.142693,-5.865061,15.058864,5.729585,190.012927,1997.065258,2011.897978,14.833623
std,21346.541880,2550.202368,692.068892,2.615227,2.804691,17.455351,9.802841,475.785043,10.050068,0.968730,10.088404
min,10.000000,0.000000,-57.000000,29.612776,-11.564592,1.000000,1.000000,0.000000,1960.000000,2001.000000,0.000000
25%,18733.000000,0.000000,0.000000,33.241978,-8.569850,5.000000,2.000000,0.000000,1995.000000,2011.000000,7.000000
50%,37376.000000,0.000000,388.000000,35.010827,-5.179386,12.000000,3.000000,30.000000,1997.000000,2012.000000,14.000000
75%,55768.000000,30.000000,1320.000000,37.259504,-3.366809,17.000000,5.000000,230.000000,2005.000000,2013.000000,16.000000
max,74249.000000,200000.000000,2777.000000,40.325016,-0.998752,99.000000,80.000000,11469.000000,2013.000000,2013.000000,53.000000


### NOTES:
* consider dropping the id cols in Train and Test as these identifiers don't really have predictive power

#### Encode categorical data as numbers???   Use  sklearn's preprocessing module called OneHotEncoder.

### Confusion Matrix goes here

In [29]:

# Alternative (AND NICER LOOKING) confusion matrix (may only work as coded for numeric targets...verify this)
##from sklearn.metrics import plot_confusion_matrix

#plot_confusion_matrix(classifier, X, y, values_format='.3g')
#plt.show()



### Consider showing most important features here

In [30]:
def plot_feature_importances(model):
    n_features = data_train.shape[1]
    plt.figure(figsize=(8,8))
    plt.barh(range(n_features), model.feature_importances_, align='center') 
    plt.yticks(np.arange(n_features), data_train.columns.values) 
    plt.xlabel('Feature importance')
    plt.ylabel('Feature')

#plot_feature_importances(tree_clf)